In [273]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [274]:
resp = requests.get('https://data.ct.gov/resource/rf3k-f8fg.json')
txt = resp.json()

In [275]:
dat_orig = pd.DataFrame(txt)

In [276]:
dat = dat_orig[dat_orig['state']=='CONNECTICUT'].copy()

dat = dat[['date', 'cases', 'deaths', 'cases_age0_9']]

cols = dat.columns.drop('date')

dat[cols] = dat[cols].apply(pd.to_numeric, errors='coerce')

dat['date'] = dat['date'].apply(pd.to_datetime)

In [277]:
sort_dat = dat.sort_values(by=['date'], inplace=False, ascending=True).copy()

In [278]:
sort_dat['daily_new_cases'] = sort_dat['cases'].diff()
sort_dat['daily_new_deaths'] = sort_dat['deaths'].diff()
sort_dat['daily_new_child_cases'] = sort_dat['cases_age0_9'].diff()

In [279]:
sort_dat['new_cases_roll3'] = sort_dat['daily_new_cases'].rolling(3).mean().round()
sort_dat['new_cases_roll7'] = sort_dat['daily_new_cases'].rolling(7).mean().round()
sort_dat['new__child_cases_roll3'] = sort_dat['daily_new_child_cases'].rolling(3).mean().round()
sort_dat['new__child_cases_roll7'] = sort_dat['daily_new_child_cases'].rolling(7).mean().round()
sort_dat['new_deaths_roll3'] = sort_dat['daily_new_deaths'].rolling(3).mean().round()
sort_dat['new_deaths_roll7'] = sort_dat['daily_new_deaths'].rolling(7).mean().round()

In [ ]:
plt.figure(num = 20, figsize=(15, 10))
plt.plot(sort_dat['date'], sort_dat['new_cases_roll7'])
# plt.plot(sort_dat['date'], sort_dat['new_cases_roll3'])
# plt.plot(sort_dat['date'], sort_dat['daily_new_cases'])
# plt.plot(sort_dat['date'], sort_dat['new__child_cases_roll7'])
# plt.plot(sort_dat['date'], sort_dat['new_deaths_roll7'])

plt.show()

In [283]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


In [284]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

def build_serv():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)
    return(service)


In [285]:
service = build_serv()

In [ ]:
def read_data(service, spreadsheet, sheet_range):
    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet, 
                                range=sheet_range).execute()
    values = result.get('values', [])
    return(values)

In [286]:
def send_data(service, dat, date_col, spreadsheet_id, sheet_range):
    send_dat = dat.copy()
    send_dat[date_col] = send_dat[date_col].dt.strftime('%Y-%m-%d')
    send_dat.fillna('', inplace=True)
    
    # How the input data should be interpreted.
    value_input_option = 'USER_ENTERED'  # USER_ENTERED to mimic Sheets entry, RAW to place data as-is

    # How the input data should be inserted.
    insert_data_option = 'OVERWRITE'  # not being used at the moment, function overwrites by default

    dfHeaders = send_dat.columns.values.tolist()
    dfHeadersArray = [dfHeaders]
    dfData = send_dat.values.tolist()

    value_range_body = {
            "majorDimension": "ROWS",
            "values": dfHeadersArray + dfData
        }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, 
                                                     range=sheet_range, 
#                                                      insertDataOption=insert_data_option,
                                                     valueInputOption=value_input_option, 
                                                     body=value_range_body)
    response = request.execute()

    return(response)





In [287]:
# Update Google Sheet with latest data
send_data(service, sort_dat, 'date', '1h8PgeXtEDaHQCmsFIcqDA3Hm_kVFRrR-jcEfVPfMwAI', 'data')

{'spreadsheetId': '1h8PgeXtEDaHQCmsFIcqDA3Hm_kVFRrR-jcEfVPfMwAI',
 'updatedRange': 'data!A1:M53',
 'updatedRows': 53,
 'updatedColumns': 13,
 'updatedCells': 689}

In [260]:
# Load alerts from Google Sheet
def pull_alerts(service, spreadsheet_id, sheet_range):
    alerts = read_data(service, spreadsheet_id, sheet_range)
    alerts = pd.DataFrame(alerts)
    new_header = alerts.iloc[0] #grab the first row for the header
    alerts = alerts[1:] #take the data less the header row
    alerts.columns = new_header #set the header row as the df header
    return(alerts)

In [261]:
alerts = pull_alerts(service, '1h8PgeXtEDaHQCmsFIcqDA3Hm_kVFRrR-jcEfVPfMwAI', 'alerts')

In [262]:
alerts

,phone,carrier,metric,threshold
1,8605599390,xfinity,new_cases_roll7,150


In [281]:
latest_day = sort_dat.iloc[len(sort_dat)-1]
# ['new_cases_roll7']

In [282]:
# for i in alerts:
#     print(i)

for row in alerts.itertuples():
#     print(row.phone)
#     print(row.Index)
    if latest_day[row.metric] <= row.threshold:
#         send message

525.0
